# GRPO Evaluation - Modular Version

**Purpose**: Evaluate trained GRPO checkpoints with proper optimization direction handling.

**Key Features**:
- ✅ **Checkpoint-first approach** - load any checkpoint to evaluate
- ✅ **Auto-detect optimization** - reads direction from checkpoint metadata
- ✅ **Independent cells** - no need to run training first
- ✅ **Correct metrics** - handles both minimization and maximization
- ✅ **Multiple modes** - single checkpoint, compare checkpoints, compare objectives

**Workflow**:
1. Select evaluation mode and checkpoints
2. Load checkpoint(s) and validate metadata
3. Generate or load test SCMs
4. Run evaluation with baselines
5. Generate visualizations with correct labels
6. Export results for analysis

## 1. Setup and Configuration

In [1]:
#!/usr/bin/env python3
"""
Cell 1: Import base components and configure environment

This cell sets up the evaluation environment.
"""

import sys
import os
from pathlib import Path
import logging
import json
import time
import subprocess
import shutil
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass

# Add project root to path
project_root = Path.cwd().parent if Path.cwd().name == "experiments" else Path.cwd()
sys.path.insert(0, str(project_root))

# Import base components
from scripts.notebooks.base_components import (
    NotebookError, CheckpointManager, SCMGenerator,
    OptimizationConfig, CheckpointMetadata, validate_environment,
    format_results_summary
)
from scripts.notebooks.config_templates import create_evaluation_config

# Core imports
import numpy as np
import jax
import jax.numpy as jnp
import jax.random as random
import pyrsistent as pyr

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s][%(name)s][%(levelname)s] - %(message)s'
)
logger = logging.getLogger(__name__)

# Validate environment
try:
    env_info = validate_environment()
    print("✅ Environment Setup Complete")
    print(f"📁 Project root: {project_root}")
    print(f"🔧 JAX devices: {env_info['jax_devices']}")
    print(f"📅 Date: {env_info['timestamp']}")
except Exception as e:
    raise NotebookError(f"Environment validation failed: {e}")

# Initialize checkpoint manager
checkpoint_dir = project_root / "checkpoints" / "grpo_training"
checkpoint_manager = CheckpointManager(checkpoint_dir)
print(f"\n📁 Checkpoint directory: {checkpoint_dir}")

INFO:2025-07-24 08:59:59,376:jax._src.xla_bridge:749: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/opt/homebrew/lib/libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/lib/libtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)
[2025-07-24 08:59:59,376][jax._src.xla_bridge][INFO] - Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/opt/homebrew/lib/libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/lib/libtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


✅ Environment Setup Complete
📁 Project root: /Users/harellidar/Documents/Imperial/Individual_Project/causal_bayes_opt
🔧 JAX devices: [CpuDevice(id=0)]
📅 Date: 2025-07-24 08:59:59

📁 Checkpoint directory: /Users/harellidar/Documents/Imperial/Individual_Project/causal_bayes_opt/checkpoints/grpo_training


In [2]:
"""
Cell 2: Configure evaluation parameters

Set evaluation mode and parameters for the run.
"""

# EVALUATION MODE SELECTION
EVALUATION_MODE = "SINGLE_CHECKPOINT"  # Options: "SINGLE_CHECKPOINT", "COMPARE_CHECKPOINTS", "COMPARE_OBJECTIVES"

# Configuration for different modes
if EVALUATION_MODE == "SINGLE_CHECKPOINT":
    # Evaluate one checkpoint against baselines
    NUM_TEST_SCMS = 10
    RUNS_PER_METHOD = 3
    INTERVENTION_BUDGET = 10
    
elif EVALUATION_MODE == "COMPARE_CHECKPOINTS":
    # Compare multiple checkpoints
    COMPARISON_COUNT = 3  # Number of checkpoints to compare
    NUM_TEST_SCMS = 5  # Fewer SCMs for faster comparison
    RUNS_PER_METHOD = 2
    INTERVENTION_BUDGET = 8
    
elif EVALUATION_MODE == "COMPARE_OBJECTIVES":
    # Compare minimization vs maximization
    NUM_TEST_SCMS = 8
    RUNS_PER_METHOD = 3
    INTERVENTION_BUDGET = 10

else:
    raise NotebookError(f"Unknown evaluation mode: {EVALUATION_MODE}")

# General settings
RANDOM_SEED = 42

print("🎯 Evaluation Configuration")
print("=" * 50)
print(f"Mode: {EVALUATION_MODE}")
print(f"Test SCMs: {NUM_TEST_SCMS}")
print(f"Runs per method: {RUNS_PER_METHOD}")
print(f"Intervention budget: {INTERVENTION_BUDGET}")
print(f"Random seed: {RANDOM_SEED}")

if EVALUATION_MODE == "COMPARE_CHECKPOINTS":
    print(f"Checkpoints to compare: {COMPARISON_COUNT}")


🎯 Evaluation Configuration
Mode: SINGLE_CHECKPOINT
Test SCMs: 10
Runs per method: 3
Intervention budget: 10
Random seed: 42


## 2. Evaluation Mode and Checkpoint Selection

In [3]:
"""
Cell 4: Select checkpoints for evaluation

This cell handles checkpoint selection based on the evaluation mode.
Uses intelligent discovery instead of hardcoded names.
"""

print("📋 Intelligent Checkpoint Selection")
print("=" * 50)

# Get available checkpoints
try:
    available_checkpoints = checkpoint_manager.list_checkpoints()
    if not available_checkpoints:
        raise NotebookError("No checkpoints found")
        
    print(f"Found {len(available_checkpoints)} total checkpoints")
    
    # Show usable checkpoints by direction
    usable_minimize = checkpoint_manager.find_usable_checkpoints('MINIMIZE')
    usable_maximize = checkpoint_manager.find_usable_checkpoints('MAXIMIZE')
    
    print(f"Usable checkpoints:")
    print(f"  MINIMIZE: {len(usable_minimize)}")
    print(f"  MAXIMIZE: {len(usable_maximize)}")
    print(f"  Total usable: {len(usable_minimize) + len(usable_maximize)}")
        
except Exception as e:
    raise NotebookError(f"Failed to analyze checkpoints: {e}")

# SELECT CHECKPOINTS BASED ON MODE
selected_checkpoints = []

if EVALUATION_MODE == "SINGLE_CHECKPOINT":
    # Find best MINIMIZE checkpoint (preferred for comparison with PARENT_SCALE)
    best_checkpoint = checkpoint_manager.find_best_checkpoint({
        'optimization_direction': 'MINIMIZE',
        'training_mode': 'QUICK'
    })
    
    if best_checkpoint:
        selected_checkpoints = [best_checkpoint]
        print(f"🎯 Selected best MINIMIZE checkpoint: {best_checkpoint.name}")
        
        # Validate the selected checkpoint
        validation = checkpoint_manager.validate_checkpoint(best_checkpoint)
        if validation['is_valid']:
            print(f"  ✅ Checkpoint is valid and ready for evaluation")
        else:
            print(f"  ⚠️ Checkpoint issues: {validation['issues']}")
    else:
        # Fallback: try any usable checkpoint
        all_usable = checkpoint_manager.find_usable_checkpoints()
        if all_usable:
            selected_checkpoints = [all_usable[0]]
            print(f"🎯 No MINIMIZE checkpoint found, using: {all_usable[0].name}")
            print(f"  Optimization: {all_usable[0].optimization_config.direction}")
        else:
            raise NotebookError("No usable checkpoints found. Please ensure checkpoints have both metadata.json and checkpoint.pkl files.")

elif EVALUATION_MODE == "COMPARE_CHECKPOINTS":
    # Get multiple usable checkpoints
    all_usable = checkpoint_manager.find_usable_checkpoints()
    comparison_count = min(COMPARISON_COUNT, len(all_usable))
    selected_checkpoints = all_usable[:comparison_count]
    print(f"📊 Selected {comparison_count} checkpoints for comparison")

elif EVALUATION_MODE == "COMPARE_OBJECTIVES":
    # Get best from each optimization direction
    best_minimize = checkpoint_manager.find_best_checkpoint({'optimization_direction': 'MINIMIZE'})
    best_maximize = checkpoint_manager.find_best_checkpoint({'optimization_direction': 'MAXIMIZE'})
    
    selected_checkpoints = []
    if best_minimize:
        selected_checkpoints.append(best_minimize)
    if best_maximize:
        selected_checkpoints.append(best_maximize)
    
    if not selected_checkpoints:
        raise NotebookError("Need checkpoints from both MINIMIZE and MAXIMIZE directions for objective comparison")
    
    print(f"🔄 Selected checkpoints for objective comparison:")
    for ckpt in selected_checkpoints:
        print(f"  - {ckpt.name} ({ckpt.optimization_config.direction})")

else:
    raise NotebookError(f"Unknown evaluation mode: {EVALUATION_MODE}")

# Final validation
if not selected_checkpoints:
    raise NotebookError("No checkpoints selected for evaluation")

print(f"✅ Final Selection ({len(selected_checkpoints)} checkpoint(s)):")
for i, checkpoint in enumerate(selected_checkpoints, 1):
    print(f"{i}. {checkpoint.name}")
    print(f"     Optimization: {checkpoint.optimization_config.direction}")
    print(f"     Training mode: {checkpoint.training_config.get('mode', 'unknown')}")
    print(f"     Path: {checkpoint.path}")
    
    # Final validation
    validation = checkpoint_manager.validate_checkpoint(checkpoint)
    if validation['is_valid']:
        print(f"     Status: ✅ Ready for evaluation")
    else:
        print(f"     Status: ❌ Issues found: {validation['issues']}")
        raise NotebookError(f"Selected checkpoint {checkpoint.name} has validation issues: {validation['issues']}")

print(f"🚀 All selected checkpoints validated and ready for evaluation\!")

<>:108: SyntaxWarning: invalid escape sequence '\!'
<>:108: SyntaxWarning: invalid escape sequence '\!'
/var/folders/2f/7z7glsfj1fd22nlr6wj56z5w0000gn/T/ipykernel_63023/2931854767.py:108: SyntaxWarning: invalid escape sequence '\!'
  print(f"🚀 All selected checkpoints validated and ready for evaluation\!")
[2025-07-24 08:59:59,391][scripts.notebooks.base_components][WARNING] - Legacy checkpoint detected - inferring optimization direction
[2025-07-24 08:59:59,391][scripts.notebooks.base_components][WARNING] - Failed to load metadata from /Users/harellidar/Documents/Imperial/Individual_Project/causal_bayes_opt/checkpoints/grpo_training/grpo_quick_minimize_20250723_094650/metadata.json: Expecting value: line 11 column 15 (char 381)
[2025-07-24 08:59:59,392][scripts.notebooks.base_components][WARNING] - Legacy checkpoint detected - inferring optimization direction
[2025-07-24 08:59:59,393][scripts.notebooks.base_components][WARNING] - Legacy checkpoint detected - inferring optimization dir

📋 Intelligent Checkpoint Selection
Found 4 total checkpoints
Usable checkpoints:
  MINIMIZE: 1
  MAXIMIZE: 2
  Total usable: 3
🎯 Selected best MINIMIZE checkpoint: grpo_quick_minimize_20250723_101252_fixed
  ✅ Checkpoint is valid and ready for evaluation
✅ Final Selection (1 checkpoint(s)):
1. grpo_quick_minimize_20250723_101252_fixed
     Optimization: MINIMIZE
     Training mode: QUICK
     Path: /Users/harellidar/Documents/Imperial/Individual_Project/causal_bayes_opt/checkpoints/grpo_training/grpo_quick_minimize_20250723_101252_fixed
     Status: ✅ Ready for evaluation
🚀 All selected checkpoints validated and ready for evaluation\!


## 3. Load and Validate Checkpoints

In [4]:
"""
Cell 3: Load checkpoint metadata and validate

This cell loads the selected checkpoints and validates their metadata.
"""

print("📥 Loading Checkpoint Metadata")
print("=" * 60)

# Store loaded checkpoint info
loaded_checkpoints = {}

for ckpt in selected_checkpoints:
    print(f"\nLoading: {ckpt.name}")
    try:
        # For now, we're using the metadata we already have
        # In production, this would load the actual model parameters
        loaded_checkpoints[ckpt.name] = {
            'metadata': ckpt,
            'optimization_config': ckpt.optimization_config,
            'training_config': ckpt.training_config,
            'model_params': None  # TODO: Load actual model parameters
        }
        
        print(f"  ✓ Optimization: {ckpt.optimization_config.direction}")
        print(f"  ✓ Training mode: {ckpt.training_config.get('mode', 'unknown')}")
        print(f"  ✓ Episodes completed: {ckpt.training_results.get('episodes_completed', 'unknown')}")
        print(f"  ✓ Duration: {ckpt.training_results.get('duration_minutes', 0):.1f} minutes")
        
        # Show reward weights if available
        if 'reward_weights' in ckpt.training_config:
            weights = ckpt.training_config['reward_weights']
            print(f"  ✓ Reward weights: opt={weights.get('optimization', 0):.1f}, "
                  f"struct={weights.get('discovery', 0):.1f}, "
                  f"eff={weights.get('efficiency', 0):.1f}")
                  
    except Exception as e:
        print(f"  ✗ Failed to load: {e}")
        raise NotebookError(f"Failed to load checkpoint {ckpt.name}: {e}")

print(f"\n✅ Loaded {len(loaded_checkpoints)} checkpoint(s) successfully")

# Check optimization compatibility for comparison modes
if EVALUATION_MODE == "COMPARE_OBJECTIVES":
    directions = [ckpt.optimization_config.direction for ckpt in selected_checkpoints]
    if len(set(directions)) == 1:
        print(f"\n⚠️ Warning: All checkpoints have same optimization direction: {directions[0]}")
        print("   Objective comparison may not be meaningful.")
    else:
        print(f"\n✅ Comparing optimization directions: {set(directions)}")

📥 Loading Checkpoint Metadata

Loading: grpo_quick_minimize_20250723_101252_fixed
  ✓ Optimization: MINIMIZE
  ✓ Training mode: QUICK
  ✓ Episodes completed: unknown
  ✓ Duration: 5.0 minutes
  ✓ Reward weights: opt=0.8, struct=0.1, eff=0.1

✅ Loaded 1 checkpoint(s) successfully


## 4. Generate Test SCMs

In [5]:
"""
Cell 4: Generate test SCMs for evaluation

Create a balanced set of test SCMs different from training.
"""

print("🔬 Generating Test SCMs")
print("=" * 60)

# Initialize SCM generator
scm_generator = SCMGenerator()

# Generate test SCMs with different seed than training
test_seed = RANDOM_SEED + 1000  # Ensure different from training

try:
    test_scms, test_metadata = scm_generator.generate_balanced_scms(
        num_scms=NUM_TEST_SCMS,
        variable_range=(3, 6),
        structure_types=['fork', 'chain', 'collider', 'mixed'],
        seed=test_seed
    )
    
    print(f"\n✅ Generated {len(test_scms)} test SCMs")
    
    # Analyze distribution
    distribution = scm_generator._summarize_distribution(test_metadata)
    print(f"\n📊 Test Set Distribution:")
    print(f"  Structure types: {distribution['structure_types']}")
    print(f"  Variable counts: {distribution['variable_counts']}")
    
    # Save test SCM metadata
    test_scm_path = project_root / "results" / "test_scms" / f"test_scms_{test_seed}.json"
    test_scm_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(test_scm_path, 'w') as f:
        json.dump({
            'metadata': test_metadata,
            'config': {
                'num_scms': len(test_scms),
                'seed': test_seed,
                'variable_range': [3, 6],
                'structure_types': ['fork', 'chain', 'collider', 'mixed']
            },
            'generated_at': datetime.now().isoformat()
        }, f, indent=2)
    
    print(f"\n💾 Saved test SCM metadata to: {test_scm_path}")
    
except Exception as e:
    raise NotebookError(f"Failed to generate test SCMs: {e}")

🔬 Generating Test SCMs


[2025-07-24 08:59:59,771][causal_bayes_opt.experiments.test_scms][INFO] - Created linear SCM with 3 variables, 2 edges, target='X1'
[2025-07-24 08:59:59,772][causal_bayes_opt.experiments.variable_scm_factory][INFO] - Generated fork SCM: 3 vars, 2 edges, target=X1
[2025-07-24 08:59:59,788][causal_bayes_opt.experiments.test_scms][INFO] - Created linear SCM with 4 variables, 3 edges, target='X2'
[2025-07-24 08:59:59,789][causal_bayes_opt.experiments.variable_scm_factory][INFO] - Generated fork SCM: 4 vars, 3 edges, target=X2
[2025-07-24 08:59:59,804][causal_bayes_opt.experiments.test_scms][INFO] - Created linear SCM with 5 variables, 4 edges, target='X2'
[2025-07-24 08:59:59,805][causal_bayes_opt.experiments.variable_scm_factory][INFO] - Generated fork SCM: 5 vars, 4 edges, target=X2
[2025-07-24 08:59:59,807][causal_bayes_opt.experiments.test_scms][INFO] - Created linear SCM with 6 variables, 5 edges, target='X3'
[2025-07-24 08:59:59,807][causal_bayes_opt.experiments.variable_scm_factory]


✅ Generated 10 test SCMs

📊 Test Set Distribution:
  Structure types: {'fork': 4, 'chain': 4, 'collider': 2}
  Variable counts: {3: 3, 4: 3, 5: 2, 6: 2}

💾 Saved test SCM metadata to: /Users/harellidar/Documents/Imperial/Individual_Project/causal_bayes_opt/results/test_scms/test_scms_1042.json


## 5. Run Evaluation

In [6]:
"""
Cell 5: Run evaluation with proper optimization handling

Evaluate checkpoints against baselines with correct metrics.
"""

print("🏁 Running Evaluation")
print("=" * 60)

# Create output directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = project_root / "results" / f"evaluation_{EVALUATION_MODE.lower()}_{timestamp}"
output_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Output directory: {output_dir}")
print(f"\nEvaluation parameters:")
print(f"  Mode: {EVALUATION_MODE}")
print(f"  Test SCMs: {NUM_TEST_SCMS}")
print(f"  Runs per method: {RUNS_PER_METHOD}")
print(f"  Intervention budget: {INTERVENTION_BUDGET}")

# Store results
evaluation_results = {}
evaluation_start_time = time.time()

# Run evaluation for each checkpoint
for ckpt in selected_checkpoints:
    print(f"\n{'='*60}")
    print(f"Evaluating: {ckpt.name}")
    print(f"Optimization: {ckpt.optimization_config.direction}")
    
    # Create checkpoint-specific output directory
    ckpt_output_dir = output_dir / ckpt.name
    ckpt_output_dir.mkdir(exist_ok=True)
    
    # Run evaluation using unified pipeline
    cmd = [
        "poetry", "run", "python",
        str(project_root / "scripts" / "unified_pipeline.py"),
        f"--checkpoint={ckpt.path}",
        f"--num-scms={min(3, NUM_TEST_SCMS)}",  # Use subset for speed
        f"--runs-per-method={RUNS_PER_METHOD}",
        f"--intervention-budget={INTERVENTION_BUDGET}",
        f"--output-dir={ckpt_output_dir}",
        f"--optimization-direction={ckpt.optimization_config.direction}"
    ]
    
    print(f"\nRunning evaluation command...")
    print(f"  (This may take a few minutes)")
    
    start_time = time.time()
    
    try:
        # Run evaluation
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode == 0:
            duration = (time.time() - start_time) / 60
            print(f"\n✅ Evaluation completed in {duration:.1f} minutes")
            
            # Load results
            results_file = ckpt_output_dir / "comparison_results.json"
            if not results_file.exists():
                # Try to find results in alternative locations
                alt_results = list(ckpt_output_dir.glob("*results*.json"))
                if alt_results:
                    results_file = alt_results[0]
            
            if results_file.exists():
                with open(results_file, 'r') as f:
                    results = json.load(f)
                
                # Store results with optimization info
                evaluation_results[ckpt.name] = {
                    'results': results,
                    'optimization_direction': ckpt.optimization_config.direction,
                    'checkpoint_metadata': ckpt.to_dict(),
                    'duration_minutes': duration
                }
                
                # Quick summary
                if 'statistical_analysis' in results:
                    print("\n📊 Quick Summary:")
                    summary = results['statistical_analysis'].get('summary_statistics', {})
                    for method, stats in list(summary.items())[:3]:  # Show top 3
                        mean_val = stats.get('target_improvement_mean', 0)
                        # Convert if needed
                        if ckpt.optimization_config.is_minimizing:
                            display_val = -mean_val  # Show actual minimized value
                        else:
                            display_val = mean_val
                        print(f"  {method}: {ckpt.optimization_config.format_improvement(display_val)}")
            else:
                print(f"⚠️ No results file found at {results_file}")
                evaluation_results[ckpt.name] = {'error': 'No results file found'}
                
        else:
            print(f"\n❌ Evaluation failed with return code {result.returncode}")
            print(f"Error: {result.stderr[:500]}...")
            evaluation_results[ckpt.name] = {'error': result.stderr}
            
    except Exception as e:
        print(f"\n❌ Evaluation failed with exception: {e}")
        evaluation_results[ckpt.name] = {'error': str(e)}

total_duration = (time.time() - evaluation_start_time) / 60

print(f"\n{'='*60}")
print(f"✅ All evaluations complete!")
print(f"  Total duration: {total_duration:.1f} minutes")
print(f"  Successful: {sum(1 for r in evaluation_results.values() if 'error' not in r)}/{len(evaluation_results)}")

🏁 Running Evaluation
📁 Output directory: /Users/harellidar/Documents/Imperial/Individual_Project/causal_bayes_opt/results/evaluation_single_checkpoint_20250724_085959

Evaluation parameters:
  Mode: SINGLE_CHECKPOINT
  Test SCMs: 10
  Runs per method: 3
  Intervention budget: 10

Evaluating: grpo_quick_minimize_20250723_101252_fixed
Optimization: MINIMIZE

Running evaluation command...
  (This may take a few minutes)

✅ Evaluation completed in 13.6 minutes

✅ All evaluations complete!
  Total duration: 13.6 minutes
  Successful: 1/1


## 6. Generate Visualizations

In [ ]:
"""
Cell 6: Generate visualizations with trajectory plots

Create comprehensive plots showing method performance over time.
"""

print("📊 Generating Visualizations")
print("=" * 60)

# Check if we have results to visualize
valid_results = {k: v for k, v in evaluation_results.items() if 'error' not in v}

if not valid_results:
    print("❌ No valid results to visualize")
else:
    # Import required visualization functions and SCM utilities
    from src.causal_bayes_opt.visualization.plots import (
        plot_baseline_comparison, plot_method_comparison
    )
    from src.causal_bayes_opt.analysis.trajectory_metrics import (
        extract_metrics_from_experiment_result
    )
    from causal_bayes_opt.data_structures.scm import get_target, get_parents
    
    # Process results for each checkpoint
    for ckpt_name, eval_data in valid_results.items():
        print(f"\n📈 Creating plots for: {ckpt_name}")
        
        results = eval_data['results']
        opt_direction = eval_data['optimization_direction']
        opt_config = OptimizationConfig(direction=opt_direction)
        
        # Extract trajectory data from method results
        plot_data = {}
        
        if 'method_results' in results:
            print(f"Found {len(results['method_results'])} methods to plot")
            
            # Get SCM metadata to find true parents
            scm_summary = results.get('scm_summary', {})
            
            for method_name, method_runs in results['method_results'].items():
                print(f"  Processing {method_name}...")
                
                # Aggregate trajectories across runs
                all_target_trajectories = []
                all_f1_trajectories = []
                all_shd_trajectories = []
                
                for run_idx, run_result in enumerate(method_runs):
                    if run_result.get('success', True):
                        # Get true parents for this run's SCM
                        true_parents = []
                        
                        # First try metadata in the result
                        if 'metadata' in run_result and 'true_parents' in run_result['metadata']:
                            true_parents = run_result['metadata']['true_parents']
                        # Then try SCM metadata
                        elif 'metadata' in run_result and 'scm_name' in run_result['metadata']:
                            scm_name = run_result['metadata']['scm_name']
                            if scm_name in scm_summary:
                                scm_info = scm_summary[scm_name]
                                if 'target_parents' in scm_info:
                                    true_parents = scm_info['target_parents']
                        # Finally try to get from SCM index
                        elif 'scm_idx' in run_result:
                            scm_idx = run_result['scm_idx']
                            # Get from test SCMs if available
                            if hasattr(test_metadata, '__iter__') and scm_idx < len(test_metadata):
                                scm_meta = test_metadata[scm_idx]
                                if 'target_parents' in scm_meta:
                                    true_parents = scm_meta['target_parents']
                        
                        # Extract trajectory using the analysis function
                        trajectory = extract_metrics_from_experiment_result(
                            run_result, 
                            true_parents
                        )
                        
                        if trajectory['target_values']:
                            all_target_trajectories.append(trajectory['target_values'])
                        if trajectory['f1_scores']:
                            all_f1_trajectories.append(trajectory['f1_scores'])
                        if trajectory['shd_values']:
                            all_shd_trajectories.append(trajectory['shd_values'])
                
                # Compute mean trajectories
                if all_target_trajectories:
                    # Pad trajectories to same length
                    max_len = max(len(t) for t in all_target_trajectories)
                    
                    # Pad and compute means
                    padded_targets = []
                    for traj in all_target_trajectories:
                        padded = list(traj) + [traj[-1]] * (max_len - len(traj))
                        padded_targets.append(padded)
                    
                    padded_f1s = []
                    for traj in all_f1_trajectories:
                        if traj:  # Check if trajectory exists
                            padded = list(traj) + [traj[-1]] * (max_len - len(traj))
                        else:
                            padded = [0.0] * max_len
                        padded_f1s.append(padded)
                        
                    padded_shds = []
                    for traj in all_shd_trajectories:
                        if traj:  # Check if trajectory exists
                            padded = list(traj) + [traj[-1]] * (max_len - len(traj))
                        else:
                            # Use number of true parents as worst-case SHD
                            worst_shd = len(true_parents) if true_parents else 3
                            padded = [worst_shd] * max_len
                        padded_shds.append(padded)
                    
                    plot_data[method_name] = {
                        'steps': list(range(1, max_len + 1)),
                        'target_mean': np.mean(padded_targets, axis=0).tolist(),
                        'target_std': np.std(padded_targets, axis=0).tolist(),
                        'f1_mean': np.mean(padded_f1s, axis=0).tolist() if padded_f1s else [],
                        'f1_std': np.std(padded_f1s, axis=0).tolist() if padded_f1s else [],
                        'shd_mean': np.mean(padded_shds, axis=0).tolist() if padded_shds else [],
                        'shd_std': np.std(padded_shds, axis=0).tolist() if padded_shds else [],
                        'n_runs': len(all_target_trajectories)
                    }
                    print(f"    ✓ Extracted {len(all_target_trajectories)} runs, {max_len} steps")
                    print(f"    ✓ F1 trajectory: {len(plot_data[method_name]['f1_mean'])} steps")
                    print(f"    ✓ SHD trajectory: {len(plot_data[method_name]['shd_mean'])} steps")
                else:
                    print(f"    ⚠️ No trajectory data found")
        
        # Create trajectory comparison plot if we have data
        if plot_data:
            print(f"\n📊 Creating trajectory plots with {len(plot_data)} methods...")
            
            # Create output directory for this checkpoint
            ckpt_plot_dir = output_dir / ckpt_name / "plots"
            ckpt_plot_dir.mkdir(parents=True, exist_ok=True)
            
            # 1. Three-panel trajectory plot
            trajectory_path = ckpt_plot_dir / "trajectory_comparison.png"
            try:
                fig = plot_baseline_comparison(
                    plot_data,
                    title=f"{ckpt_name} - {opt_config.direction} Optimization",
                    save_path=str(trajectory_path),
                    figsize=(15, 10)
                )
                plt.show()
                print(f"✅ Saved trajectory plot: {trajectory_path}")
            except Exception as e:
                print(f"⚠️ Could not create trajectory plot: {e}")
                # Fallback to simple matplotlib plot
                fig, axes = plt.subplots(1, 3, figsize=(15, 5))
                
                for method, data in plot_data.items():
                    if 'steps' in data and 'target_mean' in data:
                        color = 'red' if 'Trained' in method else 'blue'
                        label = method.replace(' + ', '\n')
                        
                        axes[0].plot(data['steps'], data['target_mean'], 
                                   label=label, color=color, marker='o', alpha=0.7)
                        
                        if 'f1_mean' in data and data['f1_mean']:
                            axes[1].plot(data['steps'], data['f1_mean'], 
                                       label=label, color=color, marker='o', alpha=0.7)
                        
                        if 'shd_mean' in data and data['shd_mean']:
                            axes[2].plot(data['steps'], data['shd_mean'], 
                                       label=label, color=color, marker='o', alpha=0.7)
                
                axes[0].set_xlabel('Intervention Step')
                axes[0].set_ylabel('Target Value')
                axes[0].set_title(f'Target Progress ({opt_config.direction})')
                axes[0].legend()
                axes[0].grid(True, alpha=0.3)
                
                axes[1].set_xlabel('Intervention Step')
                axes[1].set_ylabel('F1 Score')
                axes[1].set_title('Structure Learning (F1)')
                axes[1].legend()
                axes[1].grid(True, alpha=0.3)
                
                axes[2].set_xlabel('Intervention Step')
                axes[2].set_ylabel('SHD')
                axes[2].set_title('Structure Learning (SHD)')
                axes[2].legend()
                axes[2].grid(True, alpha=0.3)
                
                plt.suptitle(f'{ckpt_name} - Trajectory Comparison', fontsize=16)
                plt.tight_layout()
                plt.savefig(trajectory_path, dpi=300, bbox_inches='tight')
                plt.show()
                print(f"✅ Created fallback trajectory plot: {trajectory_path}")
            
            # 2. Summary bar plot
            summary_path = ckpt_plot_dir / "performance_summary.png"
            
            # Extract final values
            methods = []
            final_targets = []
            final_f1s = []
            final_shds = []
            
            for method, data in plot_data.items():
                if 'target_mean' in data and data['target_mean']:
                    methods.append(method)
                    final_targets.append(data['target_mean'][-1])
                    final_f1s.append(data['f1_mean'][-1] if data.get('f1_mean') else 0.0)
                    final_shds.append(data['shd_mean'][-1] if data.get('shd_mean') else 0.0)
            
            if methods:
                fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
                
                x = np.arange(len(methods))
                colors = ['red' if 'Trained' in m else 'blue' for m in methods]
                
                # Target values
                ax1.bar(x, final_targets, color=colors, alpha=0.7)
                ax1.set_ylabel('Final Target Value')
                ax1.set_title(f'Target Values ({opt_config.direction})')
                ax1.set_xticks(x)
                ax1.set_xticklabels([m.replace(' + ', '\n') for m in methods], 
                                   rotation=45, ha='right')
                ax1.grid(True, alpha=0.3, axis='y')
                
                # F1 scores
                ax2.bar(x, final_f1s, color=colors, alpha=0.7)
                ax2.set_ylabel('F1 Score')
                ax2.set_title('Structure Learning F1 (Higher is Better)')
                ax2.set_xticks(x)
                ax2.set_xticklabels([m.replace(' + ', '\n') for m in methods], 
                                   rotation=45, ha='right')
                ax2.grid(True, alpha=0.3, axis='y')
                
                # SHD values
                ax3.bar(x, final_shds, color=colors, alpha=0.7)
                ax3.set_ylabel('SHD')
                ax3.set_title('Structural Hamming Distance (Lower is Better)')
                ax3.set_xticks(x)
                ax3.set_xticklabels([m.replace(' + ', '\n') for m in methods], 
                                   rotation=45, ha='right')
                ax3.grid(True, alpha=0.3, axis='y')
                
                plt.suptitle(f'{ckpt_name} - Final Performance Summary', fontsize=16)
                plt.tight_layout()
                plt.savefig(summary_path, dpi=300, bbox_inches='tight')
                plt.show()
                print(f"✅ Saved summary plot: {summary_path}")
        else:
            print("⚠️ No trajectory data available for plotting")
    
    print(f"\n✅ Visualization complete!")
    print(f"📁 Plots saved to: {output_dir}")

## 7. Export Results and Summary

In [ ]:
"""
Cell 7: Export results and generate summary report

Save all results and provide clear guidance on where to find outputs.
"""

print("💾 Exporting Results")
print("=" * 60)

# Prepare export data
export_data = {
    'evaluation_config': {
        'mode': EVALUATION_MODE,
        'num_test_scms': NUM_TEST_SCMS,
        'runs_per_method': RUNS_PER_METHOD,
        'intervention_budget': INTERVENTION_BUDGET,
        'random_seed': RANDOM_SEED,
        'timestamp': timestamp
    },
    'checkpoints_evaluated': [
        {
            'name': ckpt.name,
            'optimization_direction': ckpt.optimization_config.direction,
            'path': str(ckpt.path),
            'training_mode': ckpt.training_config.get('mode', 'unknown')
        }
        for ckpt in selected_checkpoints
    ],
    'results': evaluation_results,
    'method_performance': method_performance if 'method_performance' in locals() else {},
    'duration_minutes': total_duration if 'total_duration' in locals() else 0
}

# Save JSON results
json_path = output_dir / "evaluation_results.json"

with open(json_path, 'w') as f:
    json.dump(export_data, f, indent=2)

print(f"✅ Results saved to: {json_path}")

# Generate text summary
summary_path = output_dir / "evaluation_summary.txt"
with open(summary_path, 'w') as f:
    f.write(f"GRPO Evaluation Summary\n")
    f.write(f"="*60 + "\n\n")
    
    f.write(f"Evaluation Mode: {EVALUATION_MODE}\n")
    f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}\n")
    f.write(f"Duration: {total_duration:.1f} minutes\n\n" if 'total_duration' in locals() else "\n")
    
    f.write(f"Configuration:\n")
    f.write(f"  Test SCMs: {NUM_TEST_SCMS}\n")
    f.write(f"  Runs per method: {RUNS_PER_METHOD}\n")
    f.write(f"  Intervention budget: {INTERVENTION_BUDGET}\n")
    f.write(f"  Random seed: {RANDOM_SEED}\n\n")
    
    f.write(f"Checkpoints Evaluated:\n")
    for ckpt in selected_checkpoints:
        f.write(f"  - {ckpt.name} ({ckpt.optimization_config.direction})\n")
    
    f.write(f"\nKey Findings:\n")
    f.write("-" * 60 + "\n")
    
    # Summarize results
    if 'method_performance' in locals() and method_performance:
        # Group by checkpoint
        for ckpt in selected_checkpoints:
            f.write(f"\n{ckpt.name} ({ckpt.optimization_config.direction}):\n")
            
            # Get methods for this checkpoint
            ckpt_methods = {k: v for k, v in method_performance.items() if v['checkpoint'] == ckpt.name}
            
            if ckpt_methods:
                # Sort by performance
                if ckpt.optimization_config.is_minimizing:
                    sorted_methods = sorted(ckpt_methods.items(), key=lambda x: x[1]['actual_value'])
                else:
                    sorted_methods = sorted(ckpt_methods.items(), key=lambda x: x[1]['actual_value'], reverse=True)
                
                for rank, (key, perf) in enumerate(sorted_methods[:5], 1):  # Top 5
                    marker = "*" if 'Policy' in perf['method'] or 'Trained' in perf['method'] else " "
                    f.write(f"  {rank}. {marker} {perf['method']}: {perf['display_value']}\n")
    
    if EVALUATION_MODE == "COMPARE_OBJECTIVES":
        f.write(f"\n\nOptimization Direction Insights:\n")
        f.write("-" * 60 + "\n")
        f.write("This evaluation compared minimization vs maximization objectives.\n")
        f.write("Key observation: Different optimization directions require different\n")
        f.write("reward structures and may lead to different exploration strategies.\n")
    
    f.write(f"\n\nOutput Files:\n")
    f.write(f"  Results JSON: {json_path.name}\n")
    f.write(f"  Summary: {summary_path.name}\n")
    
    # List all plots with full paths
    plot_locations = []
    
    # Check for plots in main directory
    main_plots = list(output_dir.glob("*.png"))
    for plot in main_plots:
        f.write(f"  Plot: {plot.name}\n")
        plot_locations.append(plot)
    
    # Check for plots in checkpoint subdirectories
    for ckpt in selected_checkpoints:
        ckpt_plot_dir = output_dir / ckpt.name / "plots"
        if ckpt_plot_dir.exists():
            ckpt_plots = list(ckpt_plot_dir.glob("*.png"))
            for plot in ckpt_plots:
                f.write(f"  Plot ({ckpt.name}): {plot.relative_to(output_dir)}\n")
                plot_locations.append(plot)
    
    # Also check for plots from unified pipeline
    comparison_results_path = output_dir / selected_checkpoints[0].name / "comparison_results.json"
    if comparison_results_path.exists():
        f.write(f"\n  Comparison results: {comparison_results_path.relative_to(output_dir)}\n")

print(f"✅ Summary saved to: {summary_path}")

# Display final summary with plot locations
print(f"\n🎉 Evaluation Complete!")
print(f"Mode: {EVALUATION_MODE}")
print(f"Checkpoints: {len(selected_checkpoints)}")
print(f"Duration: {total_duration:.1f} minutes" if 'total_duration' in locals() else "")
print(f"\nOutput directory: {output_dir}")

# Show plot locations clearly
if plot_locations:
    print(f"\n📊 Generated Plots ({len(plot_locations)} total):")
    for plot in plot_locations:
        print(f"  - {plot.relative_to(output_dir)}")
else:
    print(f"\n⚠️ No plots were found. Check the visualization cell output for errors.")

print(f"\n📊 Key Insights:")
if EVALUATION_MODE == "SINGLE_CHECKPOINT":
    print(f"- Evaluated {selected_checkpoints[0].name} with {selected_checkpoints[0].optimization_config.direction} objective")
    print(f"- Compare against baselines to see if training improved performance")
    print(f"- Check trajectory plots to see learning progress over time")
elif EVALUATION_MODE == "COMPARE_OBJECTIVES":
    print(f"- Compared {len(selected_checkpoints)} checkpoints with different optimization directions")
    print(f"- Minimization policies optimize for lower target values (like PARENT_SCALE)")
    print(f"- Maximization policies optimize for higher target values")
    print(f"- Check plots to see which direction performs better for your use case")

print(f"\n💡 Next steps:")
print(f"1. Review the trajectory plots showing F1, SHD, and target value progress")
print(f"2. Check the summary plots for final performance comparison")
print(f"3. Read the detailed results in {json_path.name}")
print(f"4. If plots are missing, re-run the visualization cell")

## Summary

**What we've accomplished:**
1. ✅ Loaded checkpoints with optimization direction metadata
2. ✅ Ran evaluation with proper metric handling
3. ✅ Generated visualizations that show min/max correctly
4. ✅ Exported comprehensive results

**Key improvements over original notebook:**
- Checkpoint-first approach - no need to run training
- Auto-detects optimization direction from metadata
- Correctly displays minimization vs maximization results
- All cells are independent and can be re-run
- No silent failures - explicit errors throughout

**Understanding the results:**
- For MINIMIZE checkpoints: Lower target values are better
- For MAXIMIZE checkpoints: Higher target values are better
- The plots show "(↓ better)" or "(↑ better)" to clarify
- PARENT_SCALE baseline uses minimization by default